<a href="https://colab.research.google.com/github/yijunyang/database/blob/main/4_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

*This is part of Yale course BIS638.*

- Base on `1_Relational_Database_Design.ipynb`, extend the MIMIC database with disease ontology.
- Extract synonyms of the ICD diagnoses, to realize retrieving patients with a certain diagnosis using its synonyms with SQL queries.

# Part 1 Ontology Concepts
Question: What's the difference between continuants and occurrents?

Answer: *Continuants are entities that persist through time (for example: objects, qualities), while occurrents are events in which continuants participate (for example: admissions, diagnoses).*

  



# Part 2 Ontology Design

Use [web protege](https://webprotege.stanford.edu/) to create a simple patient ontology mimicking the data model in your problem set I.

*The ontology will be shared via Protege directly.*

# Part 3 Programming

Using [OLS](https://www.ebi.ac.uk/ols/index) as an ontology resource, link the disease terms in Disease Ontology to the patient data representation (ICD diagnoses) in problem set I. Add these terms and their synonyms to your MIMIC patient data database (creating a new table). Using Python (Google Colab) and the [API](https://www.ebi.ac.uk/ols/docs/api) provided by the OLS to extract synonyms of the ICD diagnoses from Disease Ontology and populate the new table with these extracted terms and synonyms.


## 1. Preparation
- Install mySQL
- Mount google drive
- Require libraries

In [ ]:
# install mySQL
!apt install mysql-server libmysqlclient-dev
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'bis638'"
!pip install mysql-connector-python

In [36]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
# require libraries
import requests 
import pandas as pd

Now, we can create a database, using the `.sql` file generated from Vertabelo

In [5]:
# !mysql -u root -p -e "DROP DATABASE mimic3;"
!mysql -u root -p -e "CREATE DATABASE IF NOT EXISTS mimic3;\
USE mimic3;\
SOURCE /content/gdrive/My Drive/HW4_V3_create.sql;\
COMMIT;"

Enter password: 


## 2. Ontology
- Get DOID terms that have ICD9 code
- Retrieve DOID, labels, synonyms, ICD9 code
- Export the data to `.csv` files

In [7]:
# search for DOID terms that have ICD9CM codes  
# api-endpoint 
URL = "https://www.ebi.ac.uk/ols/api/search"  

# param values here 
searchterm = 'ICD9CM'
ont = "doid"
rowmax = 20 # limit to 20 terms which can be extended to 2200
fieldlist = 'iri,label,synonym,obo_id'
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'q':searchterm, 'ontology':ont, 'rows':rowmax, 'fieldList':fieldlist} 
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 
irilist = []

'''
sample code:
for d in data['response']['docs']:
  print (d['obo_id'])
  print (d['iri'])
  irilist.append(d['iri'])
  print (d['label'])
  if 'synonym' in d.keys():
    print ('synonym(s):')
    for s in d['synonym']:
      print (s)
'''

# initiate a temporary list to save the dictionaries
tmp = []
# initiate an index
i = 0

for d in data['response']['docs']:
  irilist.append(d['iri'])
  if 'synonym' in d.keys():
    for s in d['synonym']:
      # initiate a dictionary to save information
      SYN = {}
      # save DOID, label, and synonyms
      SYN['DOID'] = d['obo_id'].replace('DOID:', '')
      SYN['label'] = d['label']
      SYN['synonyms'] = s
      # save the value of dictionary in the list
      tmp.append(SYN.values())
      i = i + 1


In [8]:
# take a look at the result
tmp

[dict_values(['849', 'rheumatoid arthritis interstitial lung disease', 'Rheumatoid lung']),
 dict_values(['783', 'end stage renal disease', 'end-stage kidney disease']),
 dict_values(['783', 'end stage renal disease', 'end stage renal failure']),
 dict_values(['13227', "retinal dystrophies primarily involving Bruch's membrane", 'retinal dystrophies primarily involving Bruch membrane']),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', "Pneumonia due to Eaton's agent"]),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', 'Pneumonia due to Mycoplasma pneumoniae (disorder)']),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', 'Mycoplasmal pneumonia']),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', 'Pneumonia due to Mycoplasma pneumoniae']),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', 'Mycoplasma pneumonia']),
 dict_values(['13276', 'Mycoplasma pneumoniae pneumonia', 'cold agglutinin positive pneumonia']),
 dict_values(['13272', 'Klebsiella

In [12]:
# put the result into a pandas dataframe
df = pd.DataFrame(columns = ['DOID', 'Label', 'Synonym'], data = tmp)
df['row_id'] = range(len(df))

# rearrange columns
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols] 

# export to a csv file
df.to_csv('/content/gdrive/My Drive/mimic_iii/mimic_iii/synonym.csv', index = False)

# show the result
synonym = pd.read_csv('/content/gdrive/My Drive/mimic_iii/mimic_iii/synonym.csv')
synonym

,row_id,DOID,Label,Synonym
0,0,849,rheumatoid arthritis interstitial lung disease,Rheumatoid lung
1,1,783,end stage renal disease,end-stage kidney disease
2,2,783,end stage renal disease,end stage renal failure
3,3,13227,retinal dystrophies primarily involving Bruch'...,retinal dystrophies primarily involving Bruch ...
4,4,13276,Mycoplasma pneumoniae pneumonia,Pneumonia due to Eaton's agent
5,5,13276,Mycoplasma pneumoniae pneumonia,Pneumonia due to Mycoplasma pneumoniae (disorder)
6,6,13276,Mycoplasma pneumoniae pneumonia,Mycoplasmal pneumonia
7,7,13276,Mycoplasma pneumoniae pneumonia,Pneumonia due to Mycoplasma pneumoniae
8,8,13276,Mycoplasma pneumoniae pneumonia,Mycoplasma pneumonia
9,9,13276,Mycoplasma pneumoniae pneumonia,cold agglutinin positive pneumonia


Then, retrieve icd9 code for each iri.

In [13]:
# initiate a temporary list to save the dictionaries
tmp2 = []
# initiate an index
i = 0

'''
sample code:
for iri in irilist:
  iri2 = iri.replace(':','%253A')
  iri2 = iri2.replace('/', '%252F')
  # print (iri)
  URL2 = 'https://www.ebi.ac.uk/ols/api/ontologies/' + ont + '/terms/' + iri2
  r2 = requests.get(url = URL2)
  data2 = r2.json()
  for d2 in data2['obo_xref']:
    if d2['database'] == 'ICD9CM':
      print(iri)
      print(d2['id'])
      print(d2['id'].replace('.',''))
'''

# the code below retrieve icd9 code for each iri
for iri in irilist:
  iri2 = iri.replace(':','%253A')
  iri2 = iri2.replace('/', '%252F')
  # print (iri)
  URL2 = 'https://www.ebi.ac.uk/ols/api/ontologies/' + ont + '/terms/' + iri2
  r2 = requests.get(url = URL2)
  data2 = r2.json()
  for d2 in data2['obo_xref']:
    if d2['database'] == 'ICD9CM':
      # initiate a dictionary to save info
      ICD = {}
      ICD['DOID']= (iri.replace('http://purl.obolibrary.org/obo/DOID_', ''))
      ICD['ICD9'] = (d2['id'].replace('.',''))
      # save the value of dictionary in the list
      tmp2.append(ICD.values())
      i = i + 1

In [14]:
# take a look at the result
tmp2

[dict_values(['849', '71481']),
 dict_values(['783', '5856']),
 dict_values(['13327', '36502']),
 dict_values(['13365', '31509']),
 dict_values(['11129', '38523']),
 dict_values(['8646', '29389']),
 dict_values(['9504', '6108']),
 dict_values(['14559', '32081']),
 dict_values(['13227', '36277']),
 dict_values(['13239', '52141']),
 dict_values(['13276', '4830']),
 dict_values(['13272', '4820']),
 dict_values(['13282', '0148']),
 dict_values(['9459', '1821']),
 dict_values(['9407', '41060']),
 dict_values(['0040085', '99591']),
 dict_values(['0040084', '481']),
 dict_values(['0040083', '4831']),
 dict_values(['13145', '40511']),
 dict_values(['0040099', '7091'])]

In [15]:
# put the result into a pandas dataframe
df2 = pd.DataFrame(columns = ['DOID', 'ICD9'], data= tmp2)

# export to a csv file
df2.to_csv('/content/gdrive/My Drive/mimic_iii/mimic_iii/ICD9.csv', index = False)

# show the result
icd9 = pd.read_csv('/content/gdrive/My Drive/mimic_iii/mimic_iii/ICD9.csv')
icd9

,DOID,ICD9
0,849,71481
1,783,5856
2,13327,36502
3,13365,31509
4,11129,38523
5,8646,29389
6,9504,6108
7,14559,32081
8,13227,36277
9,13239,52141


## 3. MySQL query
- Write a SQL join query to retrieve patients with a certain diagnosis using its synonyms.

In [52]:
import mysql.connector
# establish a database connection with the load data file option set to True
cnx = mysql.connector.connect(user='root',
                password='bis638',
                host='127.0.0.1',
                database='mimic3',
                allow_local_infile=True)

# create database_cursor to perform SQL operation
db_cursor = cnx.cursor()

# execute the sql command to load a data file into a table (repeat the following line for more data files and tables)
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/PATIENTS.csv' into table PATIENTS fields terminated by ',' ignore 1 lines")
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/ADMISSIONS.csv' into table ADMISSIONS fields terminated by ',' ignore 1 lines")
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/D_ICD_DIAGNOSES.csv' into table D_ICD_DIAGNOSES fields terminated by ',' ignore 1 lines")
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/DIAGNOSES_ICD.csv' into table DIAGNOSES_ICD fields terminated by ',' ignore 1 lines")
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/ICD9.csv' into table DOID fields terminated by ',' ignore 1 lines")
db_cursor.execute("Load data local infile '/content/gdrive/My Drive/mimic_iii/mimic_iii/synonym.csv' into table SYNONYM fields terminated by ',' ignore 1 lines")

# commit the loading data transaction
db_cursor.execute("commit")

Use MySQL to retrive all the patients with a diagnoses whose synonym was 'end-stage kidney disease'.

In [51]:
!mysql -u root -p -e "USE mimic3;\
SELECT diag.subject_id, diag.hadm_id, diag.icd9_code, d.DOID, s.label, s.synonym FROM DOID d inner join SYNONYM s on d.DOID = s.DOID inner join DIAGNOSES_ICD diag on d.icd9_code = diag.icd9_code WHERE s.synonym = 'end-stage kidney disease';"


Enter password: 
+------------+---------+-----------+------+-------------------------+--------------------------+
| subject_id | hadm_id | icd9_code | DOID | label                   | synonym                  |
+------------+---------+-----------+------+-------------------------+--------------------------+
|      10029 |  132349 | 5856      | 783  | end stage renal disease | end-stage kidney disease |
|      40204 |  175237 | 5856      | 783  | end stage renal disease | end-stage kidney disease |
|      43881 |  167021 | 5856      | 783  | end stage renal disease | end-stage kidney disease |
|      43881 |  172454 | 5856      | 783  | end stage renal disease | end-stage kidney disease |
+------------+---------+-----------+------+-------------------------+--------------------------+


Another way, using db_cursor to do the query. The results are the same.

In [53]:
# Implement SQL Query
sql_statement = "SELECT diag.subject_id, diag.hadm_id, diag.icd9_code, d.DOID, s.label, s.synonym FROM DOID d \
inner join SYNONYM s on d.DOID = s.DOID \
inner join DIAGNOSES_ICD diag on d.icd9_code = diag.icd9_code \
WHERE s.synonym = 'end-stage kidney disease'"
db_cursor.execute(sql_statement)
dbresult = db_cursor.fetchall()
for r in dbresult:
  print(r)
db_cursor.close()
cnx.close()

(10029, 132349, '5856', '783', 'end stage renal disease', 'end-stage kidney disease')
(40204, 175237, '5856', '783', 'end stage renal disease', 'end-stage kidney disease')
(43881, 167021, '5856', '783', 'end stage renal disease', 'end-stage kidney disease')
(43881, 172454, '5856', '783', 'end stage renal disease', 'end-stage kidney disease')
